In [2]:
#importing libraries
import re
import pandas as pd
import numpy as np
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
import xgboost,textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers


In [3]:
trainDF=pd.read_csv("clean.csv",encoding='mac_roman', na_filter=True, na_values='[]')
trainDF.dropna(inplace=True)

In [4]:
trainDF

,Unnamed: 0,content,Labels,text_clean,text_tokens,text_tokens_lemma
0,0,Turns out the cop did not even bake that fucki...,1,turns cop even bake fucking cake,"['turns', 'cop', 'even', 'bake', 'fucking', 'c...","['turn', 'cop', 'even', 'bake', 'fucking', 'ca..."
1,1,@realDonaldTrump Hey Don! You up? Donnnnaaald!...,0,hey donnnnaaald awake,"['hey', 'donnnnaaald', 'awake']","['hey', 'donnnnaaald', 'awake']"
2,2,Man drowned in Nichols home is 5th hurricane d...,1,man drowned nichols home th hurricane death sc...,"['man', 'drowned', 'nichols', 'home', 'th', 'h...","['man', 'drowned', 'nichols', 'home', 'th', 'h..."
3,3,It‚Äö√Ñ√¥s time to phase immigration down so w...,1,time phase immigration phase americans back,"['time', 'phase', 'immigration', 'phase', 'ame...","['time', 'phase', 'immigration', 'phase', 'ame..."
4,4,Calm is a superpower. ‚Äö√∫√• https://t.co/B...,1,calm superpower,"['calm', 'superpower']","['calm', 'superpower']"
...,...,...,...,...,...,...
83413,83413,SWING STATE SWING: New poll shows Trump +4 in ...,1,swing state swing new poll shows trump pa,"['swing', 'state', 'swing', 'new', 'poll', 'sh...","['swing', 'state', 'swing', 'new', 'poll', 'sh..."
83414,83414,BLOOP! \| On Black Women As Birth Of A Nation‚...,1,bloop black women birth nations scapegoats rol...,"['bloop', 'black', 'women', 'birth', 'nations'...","['bloop', 'black', 'woman', 'birth', 'nation',..."
83415,83415,Bob Dylan wins 2016 Nobel Prize in literature ...,1,bob dylan wins nobel prize literature,"['bob', 'dylan', 'wins', 'nobel', 'prize', 'li...","['bob', 'dylan', 'win', 'nobel', 'prize', 'lit..."
83416,83416,"ha scream, annoyed, screaming, enough, yell, s...",1,ha scream annoyed screaming enough yell stop a...,"['ha', 'scream', 'annoyed', 'screaming', 'enou...","['ha', 'scream', 'annoyed', 'screaming', 'enou..."


In [5]:
X=trainDF['text_tokens_lemma']
Y=trainDF['Labels']

In [6]:
X

0        ['turn', 'cop', 'even', 'bake', 'fucking', 'ca...
1                          ['hey', 'donnnnaaald', 'awake']
2        ['man', 'drowned', 'nichols', 'home', 'th', 'h...
3        ['time', 'phase', 'immigration', 'phase', 'ame...
4                                   ['calm', 'superpower']
                               ...                        
83413    ['swing', 'state', 'swing', 'new', 'poll', 'sh...
83414    ['bloop', 'black', 'woman', 'birth', 'nation',...
83415    ['bob', 'dylan', 'win', 'nobel', 'prize', 'lit...
83416    ['ha', 'scream', 'annoyed', 'screaming', 'enou...
83417                   ['need', 'help', 'finding', 'job']
Name: text_tokens_lemma, Length: 82661, dtype: object

In [7]:
Y

0        1
1        0
2        1
3        1
4        1
        ..
83413    1
83414    1
83415    1
83416    1
83417    1
Name: Labels, Length: 82661, dtype: int64

In [8]:
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(X, Y,test_size=0.2)

In [9]:
print(train_x.shape,valid_x.shape,train_y.shape,valid_y.shape)

(66128,) (16533,) (66128,) (16533,)


In [10]:
train_x

20960    ['explanation', 'expected', 'im', 'glad', 'won...
6921     ['mainstream', 'medium', 'doesnt', 'hold', 'hi...
47761    ['tune', 'ogradio', 'w', 'radio', 'pm', 'et', ...
54465    ['trump', 'tax', 'record', 'obtained', 'time',...
46666    ['welp', 'proof', 'rigged', 'debate', 'schedul...
                               ...                        
41920    ['u', 'lived', 'thru', 'year', 'scandal', 'kno...
11339    ['nice', 'afternoon', 'vega', 'meeting', 'fan'...
28839    ['chicago', 'teacher', 'union', 'vote', 'appro...
31908    ['budweiser', 'study', 'show', 'american', 'wa...
82785    ['thank', 'much', 'shoutout', 'lovetrumpshate'...
Name: text_tokens_lemma, Length: 66128, dtype: object

## Count Vectors as features

In [11]:
# create a count vectorizer object 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(trainDF['text_tokens_lemma'])

# transform the training and validation data using count vectorizer object
xtrain_count =  count_vect.transform(train_x)
xvalid_count =  count_vect.transform(valid_x)

In [12]:
xtrain_count

<66128x44572 sparse matrix of type '<class 'numpy.int64'>'
	with 524711 stored elements in Compressed Sparse Row format>

In [13]:
xvalid_count

<16533x44572 sparse matrix of type '<class 'numpy.int64'>'
	with 131962 stored elements in Compressed Sparse Row format>

## TF-IDF Vectors as features

In [14]:
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=8000)
tfidf_vect.fit(trainDF['text_tokens_lemma'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(valid_x)

# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=8000)
tfidf_vect_ngram.fit(trainDF['text_tokens_lemma'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)

# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=8000)
tfidf_vect_ngram_chars.fit(trainDF['text_tokens_lemma'])
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(train_x) 
xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(valid_x) 

C:\Users\shrus\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:501: UserWarning: The parameter 'token_pattern' will not be used since 'analyzer' != 'word'
  warnings.warn("The parameter 'token_pattern' will not be used"


## Word Embeddings

In [15]:
'''# load the pre-trained word-embedding vectors 
embeddings_index = {}
for i, line in enumerate(open('data/wiki-news-300d-1M.vec')):
    values = line.split()
    embeddings_index[values[0]] = numpy.asarray(values[1:], dtype='float32')

# create a tokenizer 
token = text.Tokenizer()
token.fit_on_texts(trainDF['text_tokens_lemma'])
word_index = token.word_index

# convert text to sequence of tokens and pad them to ensure equal length vectors 
train_seq_x = sequence.pad_sequences(token.texts_to_sequences(train_x), maxlen=70)
valid_seq_x = sequence.pad_sequences(token.texts_to_sequences(valid_x), maxlen=70)

# create token-embedding mapping
embedding_matrix = numpy.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector'''

"# load the pre-trained word-embedding vectors \nembeddings_index = {}\nfor i, line in enumerate(open('data/wiki-news-300d-1M.vec')):\n    values = line.split()\n    embeddings_index[values[0]] = numpy.asarray(values[1:], dtype='float32')\n\n# create a tokenizer \ntoken = text.Tokenizer()\ntoken.fit_on_texts(trainDF['text_tokens_lemma'])\nword_index = token.word_index\n\n# convert text to sequence of tokens and pad them to ensure equal length vectors \ntrain_seq_x = sequence.pad_sequences(token.texts_to_sequences(train_x), maxlen=70)\nvalid_seq_x = sequence.pad_sequences(token.texts_to_sequences(valid_x), maxlen=70)\n\n# create token-embedding mapping\nembedding_matrix = numpy.zeros((len(word_index) + 1, 300))\nfor word, i in word_index.items():\n    embedding_vector = embeddings_index.get(word)\n    if embedding_vector is not None:\n        embedding_matrix[i] = embedding_vector"

## MODEL

In [16]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    return metrics.accuracy_score(predictions, valid_y)

## NB

In [17]:
# Naive Bayes on Count Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_count, train_y, xvalid_count)
print ("NB, Count Vectors: ", accuracy)

# Naive Bayes on Word Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf, train_y, xvalid_tfidf)
print ("NB, WordLevel TF-IDF: ", accuracy)

# Naive Bayes on Ngram Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print ("NB, N-Gram Vectors: ", accuracy)

# Naive Bayes on Character Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print ("NB, CharLevel Vectors: ", accuracy)


NB, Count Vectors:  0.9357648339684268
NB, WordLevel TF-IDF:  0.937276961229057
NB, N-Gram Vectors:  0.9372164761386318
NB, CharLevel Vectors:  0.9309260267344099


##  LR

In [18]:
# Linear Classifier on Count Vectors
accuracy = train_model(linear_model.LogisticRegression(max_iter=500), xtrain_count, train_y, xvalid_count)
print ("LR, Count Vectors: ", accuracy)

# Linear Classifier on Word Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(max_iter=500), xtrain_tfidf, train_y, xvalid_tfidf)
print ("LR, WordLevel TF-IDF: ", accuracy)

# Linear Classifier on Ngram Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(max_iter=500), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print ("LR, N-Gram Vectors: ", accuracy)

# Linear Classifier on Character Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(max_iter=500), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print ("LR, CharLevel Vectors: ", accuracy)

LR, Count Vectors:  0.9500998003992016
LR, WordLevel TF-IDF:  0.9452005080747595
LR, N-Gram Vectors:  0.9363696848726789
LR, CharLevel Vectors:  0.9390915139418133


## RF

In [19]:
'''# RF on Count Vectors
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_count, train_y, xvalid_count)
print ("RF, Count Vectors: ", accuracy)

#RF on Word Level TF IDF Vectors
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_tfidf, train_y, xvalid_tfidf)
print ("RF, WordLevel TF-IDF: ", accuracy)

# RF on Ngram Level TF IDF Vectors
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_tfidf_ngram, train_y, xvalid_count)
print ("RF, Count Vectors: ", accuracy)

# RF on Character Level TF IDF Vectors
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf)
print ("RF, WordLevel TF-IDF: ", accuracy)'''

'# RF on Count Vectors\naccuracy = train_model(ensemble.RandomForestClassifier(), xtrain_count, train_y, xvalid_count)\nprint ("RF, Count Vectors: ", accuracy)\n\n#RF on Word Level TF IDF Vectors\naccuracy = train_model(ensemble.RandomForestClassifier(), xtrain_tfidf, train_y, xvalid_tfidf)\nprint ("RF, WordLevel TF-IDF: ", accuracy)\n\n# RF on Ngram Level TF IDF Vectors\naccuracy = train_model(ensemble.RandomForestClassifier(), xtrain_tfidf_ngram, train_y, xvalid_count)\nprint ("RF, Count Vectors: ", accuracy)\n\n# RF on Character Level TF IDF Vectors\naccuracy = train_model(ensemble.RandomForestClassifier(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf)\nprint ("RF, WordLevel TF-IDF: ", accuracy)'

## SVM

In [20]:
'''# SVM on Count Vectors
accuracy = train_model(svm.SVC(), xtrain_count, train_y, xvalid_count)
print ("SVM, Count Vectors: ", accuracy)

# SVM on Word Level TF IDF Vectors
accuracy = train_model(svm.SVC(), xtrain_tfidf, train_y, xvalid_tfidf)
print ("SVM, WordLevel TF-IDF: ", accuracy)

# SVM on Ngram Level TF IDF Vectors
accuracy = train_model(svm.SVC(), xtrain_tfidf_ngram, train_y, xvalid_count)
print ("SVM, Count Vectors: ", accuracy)

# SVM on Character Level TF IDF Vectors
accuracy = train_model(svm.SVC(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf)
print ("SVM, WordLevel TF-IDF: ", accuracy)'''

'# SVM on Count Vectors\naccuracy = train_model(svm.SVC(), xtrain_count, train_y, xvalid_count)\nprint ("SVM, Count Vectors: ", accuracy)\n\n# SVM on Word Level TF IDF Vectors\naccuracy = train_model(svm.SVC(), xtrain_tfidf, train_y, xvalid_tfidf)\nprint ("SVM, WordLevel TF-IDF: ", accuracy)\n\n# SVM on Ngram Level TF IDF Vectors\naccuracy = train_model(svm.SVC(), xtrain_tfidf_ngram, train_y, xvalid_count)\nprint ("SVM, Count Vectors: ", accuracy)\n\n# SVM on Character Level TF IDF Vectors\naccuracy = train_model(svm.SVC(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf)\nprint ("SVM, WordLevel TF-IDF: ", accuracy)'

## LSTM

In [21]:
'''def create_rnn_lstm():
    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the LSTM Layer
    lstm_layer = layers.LSTM(100)(embedding_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(lstm_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    
    return model

classifier = create_rnn_lstm()
accuracy = train_model(classifier, train_seq_x, train_y, valid_seq_x, is_neural_net=True)
print ("RNN-LSTM, Word Embeddings",  accuracy)'''

'def create_rnn_lstm():\n    # Add an Input Layer\n    input_layer = layers.Input((70, ))\n\n    # Add the word embedding Layer\n    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)\n    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)\n\n    # Add the LSTM Layer\n    lstm_layer = layers.LSTM(100)(embedding_layer)\n\n    # Add the output Layers\n    output_layer1 = layers.Dense(50, activation="relu")(lstm_layer)\n    output_layer1 = layers.Dropout(0.25)(output_layer1)\n    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)\n\n    # Compile the model\n    model = models.Model(inputs=input_layer, outputs=output_layer2)\n    model.compile(optimizer=optimizers.Adam(), loss=\'binary_crossentropy\')\n    \n    return model\n\nclassifier = create_rnn_lstm()\naccuracy = train_model(classifier, train_seq_x, train_y, valid_seq_x, is_neural_net=True)\nprint ("RNN-LSTM, Word Embeddings",  accurac

## BEST MODEL (selected on the basis of accuracy on validation set and time taken to train the model)

LR on Count Vectors is the best model WITH ACCURACY OF approx 94.8% on validation set

Also SVM on tfidF gives an ACCURACY OF 94.82% but takes longer time to train an predict 



In [22]:
from sklearn.metrics import confusion_matrix, accuracy_score
loreg=linear_model.LogisticRegression(max_iter=200)# initialize the model
loreg.fit(xtrain_count, train_y) # fit he model
y_pred=loreg.predict(xvalid_count) # now predict
cm = confusion_matrix(valid_y, y_pred)
print(cm)
accuracy_score(valid_y, y_pred)    

[[  808   648]
 [  177 14900]]


0.9500998003992016